In [1]:
import pandas as pd
from tensorflow.keras.metrics import CategoricalAccuracy
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
import torch
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L

## Read and sort training data

In [2]:
path = "/Users/udyansachdev/Downloads/Recipes5k/annotations"

In [3]:
file_path = path + "/" + "train_images.txt"

# Read the text file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Create DataFrame from lines
y_train_id = pd.DataFrame(lines, columns=["id"])

In [4]:
y_train_id

,id
0,apple_pie/20_homemade_apple_pie_hostedLargeUrl...
1,apple_pie/43_homestyle_apple_pie_hostedLargeUr...
2,apple_pie/28_apple_pie_hostedLargeUrl.jpg\n
3,apple_pie/39_classic_apple_pie_hostedLargeUrl....
4,apple_pie/13_mug_apple_pie_hostedLargeUrl.jpg\n
...,...
3404,chocolate_ice_cream/12_caramelized_white_choco...
3405,chocolate_ice_cream/32_chocolate_ice_cream_hos...
3406,chocolate_ice_cream/8_milkiest_chocolate_ice_c...
3407,chocolate_ice_cream/7_chocolate_ice_cream_host...


In [5]:
file_path = path + "/" + "train_labels.txt"

# Read the text file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Create DataFrame from lines
y_train_label = pd.DataFrame(lines, columns=["label"])

In [6]:
y_train_label["label"] = y_train_label["label"].str.strip()
y_train_label

,label
0,69
1,91
2,77
3,88
4,62
...,...
3404,4788
3405,4808
3406,4784
3407,4783


In [7]:
y_train = pd.concat([y_train_id, y_train_label], axis=1)

In [8]:
y_train["label"] = y_train["label"].astype(int)

In [9]:
y_train

,id,label
0,apple_pie/20_homemade_apple_pie_hostedLargeUrl...,69
1,apple_pie/43_homestyle_apple_pie_hostedLargeUr...,91
2,apple_pie/28_apple_pie_hostedLargeUrl.jpg\n,77
3,apple_pie/39_classic_apple_pie_hostedLargeUrl....,88
4,apple_pie/13_mug_apple_pie_hostedLargeUrl.jpg\n,62
...,...,...
3404,chocolate_ice_cream/12_caramelized_white_choco...,4788
3405,chocolate_ice_cream/32_chocolate_ice_cream_hos...,4808
3406,chocolate_ice_cream/8_milkiest_chocolate_ice_c...,4784
3407,chocolate_ice_cream/7_chocolate_ice_cream_host...,4783


### Read ingredients

In [10]:
file_path = path + "/" + "ingredients_simplified_Recipes5k.txt"

# Read the text file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Create DataFrame from lines
ingredients = pd.DataFrame(lines, columns=["ingredients"])

In [11]:
ingredients["ingredients"] = ingredients["ingredients"].str.strip()
ingredients

,ingredients
0,"onion,flour,baking,salt,egg,milk,bread,coconut..."
1,"onion,panko,cornmeal,paprika,onion,garlic,caye..."
2,"onion,flour,baking,egg,bread,oil"
3,"oil,cake,onion"
4,"nut,onion,flour,egg,celery,baking,ale"
...,...
4821,"chocolate,milk,cocoa,egg,sugar,water,whipping"
4822,"& half,milk,sugar,salt,dutch process cocoa,cho..."
4823,"cocoa,coffee,sugar,chocolate,milk,corn starch,..."
4824,"cocoa,sugar,brown sugar,milk,vanilla,salt"


In [12]:
special_characters = [
    "@",
    "#",
    "$",
    "%",
    "&",
    "*",
    "+",
    "-",
    "/",
    "\\",
    ":",
    ";",
    "<",
    "=",
    ">",
    "?",
    "[",
    "]",
    "^",
    "_",
    "`",
    "{",
    "|",
    "}",
    "~",
    "!",
    '"',
    "'",
    "(",
    ")",
    ".",
]

for char in special_characters:
    ingredients["ingredients"] = ingredients["ingredients"].str.replace(char, "")

ingredients

,ingredients
0,"onion,flour,baking,salt,egg,milk,bread,coconut..."
1,"onion,panko,cornmeal,paprika,onion,garlic,caye..."
2,"onion,flour,baking,egg,bread,oil"
3,"oil,cake,onion"
4,"nut,onion,flour,egg,celery,baking,ale"
...,...
4821,"chocolate,milk,cocoa,egg,sugar,water,whipping"
4822,"half,milk,sugar,salt,dutch process cocoa,choc..."
4823,"cocoa,coffee,sugar,chocolate,milk,corn starch,..."
4824,"cocoa,sugar,brown sugar,milk,vanilla,salt"


In [13]:
ingredients["label"] = ingredients.index

### Merge dataframe

In [14]:
y_train_final = y_train.merge(ingredients, how="left", on="label")

In [15]:
y_train_final["id"] = y_train_final["id"].str.strip()
y_train_final

,id,label,ingredients
0,apple_pie/20_homemade_apple_pie_hostedLargeUrl...,69,"flour,salt,oil,cold water,apple,sugar,cinnamon..."
1,apple_pie/43_homestyle_apple_pie_hostedLargeUr...,91,"shell,pie,sugar,flour,cinnamon,apple,lemon,but..."
2,apple_pie/28_apple_pie_hostedLargeUrl.jpg,77,"apple,lemon,sugar,flour,cinnamon,nut,butter,sa..."
3,apple_pie/39_classic_apple_pie_hostedLargeUrl.jpg,88,"pie,apple,sugar,corn starch,cinnamon,lemon,but..."
4,apple_pie/13_mug_apple_pie_hostedLargeUrl.jpg,62,"apple,brown sugar,butter,cinnamon,allspice,nut..."
...,...,...,...
3404,chocolate_ice_cream/12_caramelized_white_choco...,4788,"chocolate,milk,sugar,salt,egg"
3405,chocolate_ice_cream/32_chocolate_ice_cream_hos...,4808,"chocolate,chocolate,whipping,milk,cheese,sugar..."
3406,chocolate_ice_cream/8_milkiest_chocolate_ice_c...,4784,"milk,corn starch,chocolate,milk,sugar,corn syr..."
3407,chocolate_ice_cream/7_chocolate_ice_cream_host...,4783,"whipping,milk,egg,sugar,cocoa,vanilla,chocolat..."


In [16]:
"""
# Split ingredients into individual items
y_train_final["ingredients"] = y_train_final["ingredients"].str.split(",")

# One-hot encode the 'ingredients' column
encoded_data = y_train_final["ingredients"].str.join("|").str.get_dummies()

# Concatenate the original DataFrame with the encoded DataFrame
data_encoded = pd.concat([y_train_final, encoded_data], axis=1)
"""

'\n# Split ingredients into individual items\ny_train_final["ingredients"] = y_train_final["ingredients"].str.split(",")\n\n# One-hot encode the \'ingredients\' column\nencoded_data = y_train_final["ingredients"].str.join("|").str.get_dummies()\n\n# Concatenate the original DataFrame with the encoded DataFrame\ndata_encoded = pd.concat([y_train_final, encoded_data], axis=1)\n'

In [17]:
# data_encoded

## Loading and preprocessing the training data

In [18]:
# data_encoded["id"] = data_encoded["id"].str.strip()

In [19]:
"""
train_image = []
for i in data_encoded["id"]:
    img = image.load_img(
        ("/Users/udyansachdev/Downloads/Recipes5k/images" + "//" + i),
        target_size=(400, 400, 3),
    )
    img = image.img_to_array(img)
    img = img / 255
    train_image.append(img)
X = np.array(train_image)
"""

'\ntrain_image = []\nfor i in data_encoded["id"]:\n    img = image.load_img(\n        ("/Users/udyansachdev/Downloads/Recipes5k/images" + "//" + i),\n        target_size=(400, 400, 3),\n    )\n    img = image.img_to_array(img)\n    img = img / 255\n    train_image.append(img)\nX = np.array(train_image)\n'

In [20]:
# X.shape

In [21]:
"""
plt.imshow(X[2000])
data_encoded["ingredients"][2000]
"""

'\nplt.imshow(X[2000])\ndata_encoded["ingredients"][2000]\n'

In [22]:
# y = np.array(data_encoded.drop(["id", "label", "ingredients"], axis=1))

In [23]:
# y.shape

## Read and sort validation data

In [24]:
file_path = path + "/" + "val_images.txt"

# Read the text file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Create DataFrame from lines
y_val_id = pd.DataFrame(lines, columns=["id"])

In [25]:
y_val_id

,id
0,apple_pie/42_easy_apple_pie_hostedLargeUrl.jpg\n
1,apple_pie/49_raw_apple_pie_hostedLargeUrl.jpg\n
2,apple_pie/11_classic_apple_pie_hostedLargeUrl....
3,apple_pie/24_apple_pie_hostedLargeUrl.jpg\n
4,apple_pie/10_apple_pie_hostedLargeUrl.jpg\n
...,...
629,chocolate_ice_cream/18_very_chocolate_ice_crea...
630,chocolate_ice_cream/36_chocolate_ice_cream_hos...
631,chocolate_ice_cream/43_chocolate_ice_cream_hos...
632,chocolate_ice_cream/27_basic_chocolate_ice_cre...


In [26]:
file_path = path + "/" + "val_labels.txt"

# Read the text file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Create DataFrame from lines
y_val_label = pd.DataFrame(lines, columns=["label"])

In [27]:
y_val_label["label"] = y_val_label["label"].str.strip()
y_val_label

,label
0,90
1,97
2,60
3,73
4,59
...,...
629,4794
630,4812
631,4819
632,4803


In [28]:
y_val = pd.concat([y_val_id, y_val_label], axis=1)
y_val["label"] = y_val["label"].astype(int)
y_val["id"] = y_val["id"].str.strip()

y_val

,id,label
0,apple_pie/42_easy_apple_pie_hostedLargeUrl.jpg,90
1,apple_pie/49_raw_apple_pie_hostedLargeUrl.jpg,97
2,apple_pie/11_classic_apple_pie_hostedLargeUrl.jpg,60
3,apple_pie/24_apple_pie_hostedLargeUrl.jpg,73
4,apple_pie/10_apple_pie_hostedLargeUrl.jpg,59
...,...,...
629,chocolate_ice_cream/18_very_chocolate_ice_crea...,4794
630,chocolate_ice_cream/36_chocolate_ice_cream_hos...,4812
631,chocolate_ice_cream/43_chocolate_ice_cream_hos...,4819
632,chocolate_ice_cream/27_basic_chocolate_ice_cre...,4803


In [29]:
y_val_final = y_val.merge(ingredients, how="left", on="label")

In [30]:
y_val_final

,id,label,ingredients
0,apple_pie/42_easy_apple_pie_hostedLargeUrl.jpg,90,"nut,cinnamon,apple,sugar,flour,cinnamon,nut,bu..."
1,apple_pie/49_raw_apple_pie_hostedLargeUrl.jpg,97,"apple,lemon,cinnamon,date,nut"
2,apple_pie/11_classic_apple_pie_hostedLargeUrl.jpg,60,"pie,apple,sugar,flour,cinnamon,salt,nut,lemon"
3,apple_pie/24_apple_pie_hostedLargeUrl.jpg,73,"apple,sugar,water,butter,caster sugar,flour,fl..."
4,apple_pie/10_apple_pie_hostedLargeUrl.jpg,59,"pie,corn starch,equal sweetener,cinnamon,nut,s..."
...,...,...,...
629,chocolate_ice_cream/18_very_chocolate_ice_crea...,4794,"sugar,milk,salt,cocoa,egg,chocolate,vanilla"
630,chocolate_ice_cream/36_chocolate_ice_cream_hos...,4812,"whipping,milk,sugar,cocoa,vanilla"
631,chocolate_ice_cream/43_chocolate_ice_cream_hos...,4819,"cocoa, half,egg,sugar,vanilla,oreou00ae cookies"
632,chocolate_ice_cream/27_basic_chocolate_ice_cre...,4803,"sugar,egg,egg,cocoa,milk,vanilla"


##  Concat Dataframe of y_train and y_val

In [31]:
y = pd.concat([y_train_final, y_val_final], axis=0, ignore_index=True)

In [32]:
y

,id,label,ingredients
0,apple_pie/20_homemade_apple_pie_hostedLargeUrl...,69,"flour,salt,oil,cold water,apple,sugar,cinnamon..."
1,apple_pie/43_homestyle_apple_pie_hostedLargeUr...,91,"shell,pie,sugar,flour,cinnamon,apple,lemon,but..."
2,apple_pie/28_apple_pie_hostedLargeUrl.jpg,77,"apple,lemon,sugar,flour,cinnamon,nut,butter,sa..."
3,apple_pie/39_classic_apple_pie_hostedLargeUrl.jpg,88,"pie,apple,sugar,corn starch,cinnamon,lemon,but..."
4,apple_pie/13_mug_apple_pie_hostedLargeUrl.jpg,62,"apple,brown sugar,butter,cinnamon,allspice,nut..."
...,...,...,...
4038,chocolate_ice_cream/18_very_chocolate_ice_crea...,4794,"sugar,milk,salt,cocoa,egg,chocolate,vanilla"
4039,chocolate_ice_cream/36_chocolate_ice_cream_hos...,4812,"whipping,milk,sugar,cocoa,vanilla"
4040,chocolate_ice_cream/43_chocolate_ice_cream_hos...,4819,"cocoa, half,egg,sugar,vanilla,oreou00ae cookies"
4041,chocolate_ice_cream/27_basic_chocolate_ice_cre...,4803,"sugar,egg,egg,cocoa,milk,vanilla"


## Loading and preprocessing the testing data

In [33]:
file_path = path + "/" + "test_images.txt"

# Read the text file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Create DataFrame from lines
y_test_id = pd.DataFrame(lines, columns=["id"])

In [34]:
y_test_id["id"] = y_test_id["id"].str.strip()
y_test_id

,id
0,apple_pie/41_apple_slab_pie_hostedLargeUrl.jpg
1,apple_pie/36_vermont_apple_slab_pie_hostedLarg...
2,apple_pie/31_skinny_apple_pie_hostedLargeUrl.jpg
3,apple_pie/2_perfect_apple_pie_hostedLargeUrl.jpg
4,apple_pie/29_traditional_apple_pie_hostedLarge...
...,...
778,chocolate_ice_cream/41_homemade_chocolate_ice_...
779,chocolate_ice_cream/29_chocolate_ice_cream_hos...
780,"chocolate_ice_cream/39_vanilla_bean,_white_cho..."
781,chocolate_ice_cream/13_homemade_chocolate_ice_...


In [35]:
file_path = path + "/" + "test_labels.txt"

# Read the text file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Create DataFrame from lines
y_test_label = pd.DataFrame(lines, columns=["label"])

In [36]:
y_test_label["label"] = y_test_label["label"].str.strip()
y_test_label

,label
0,89
1,85
2,80
3,51
4,78
...,...
778,4817
779,4805
780,4815
781,4789


In [37]:
y_test = pd.concat([y_test_id, y_test_label], axis=1)

In [38]:
y_test["label"] = y_test["label"].astype(int)

In [39]:
y_test

,id,label
0,apple_pie/41_apple_slab_pie_hostedLargeUrl.jpg,89
1,apple_pie/36_vermont_apple_slab_pie_hostedLarg...,85
2,apple_pie/31_skinny_apple_pie_hostedLargeUrl.jpg,80
3,apple_pie/2_perfect_apple_pie_hostedLargeUrl.jpg,51
4,apple_pie/29_traditional_apple_pie_hostedLarge...,78
...,...,...
778,chocolate_ice_cream/41_homemade_chocolate_ice_...,4817
779,chocolate_ice_cream/29_chocolate_ice_cream_hos...,4805
780,"chocolate_ice_cream/39_vanilla_bean,_white_cho...",4815
781,chocolate_ice_cream/13_homemade_chocolate_ice_...,4789


### Merging with ingredients


In [40]:
y_test_final = y_test.merge(ingredients, how="left", on="label")

In [41]:
y_test_final

,id,label,ingredients
0,apple_pie/41_apple_slab_pie_hostedLargeUrl.jpg,89,"puff sheets,butter,apple,brown sugar,cinnamon,egg"
1,apple_pie/36_vermont_apple_slab_pie_hostedLarg...,85,"pie,apple,cornflakes,sugar,cinnamon,sugar,mapl..."
2,apple_pie/31_skinny_apple_pie_hostedLargeUrl.jpg,80,"apple,sugar,cinnamon,whipped"
3,apple_pie/2_perfect_apple_pie_hostedLargeUrl.jpg,51,"pie,apple,sugar,flour,cinnamon,salt,nut,lemon"
4,apple_pie/29_traditional_apple_pie_hostedLarge...,78,"apple,butter,maple,sugar,cinnamon,apple,flour,pie"
...,...,...,...
778,chocolate_ice_cream/41_homemade_chocolate_ice_...,4817,"chocolate,egg,sugar,vanilla,chocolate,milk"
779,chocolate_ice_cream/29_chocolate_ice_cream_hos...,4805,"whipping,condensed milk,chocolate"
780,"chocolate_ice_cream/39_vanilla_bean,_white_cho...",4815,"milk,sugar,whipping,vanilla,beans,chocolate,or..."
781,chocolate_ice_cream/13_homemade_chocolate_ice_...,4789,"condensed milk,milk,chocolate,cocoa,corn starc..."


## Merging Training and Testing Dataframe

### Ensuring the split have uniform y one hot encoded 

In [42]:
data = pd.concat([y, y_test_final], axis=0, ignore_index=True)

In [43]:
data

,id,label,ingredients
0,apple_pie/20_homemade_apple_pie_hostedLargeUrl...,69,"flour,salt,oil,cold water,apple,sugar,cinnamon..."
1,apple_pie/43_homestyle_apple_pie_hostedLargeUr...,91,"shell,pie,sugar,flour,cinnamon,apple,lemon,but..."
2,apple_pie/28_apple_pie_hostedLargeUrl.jpg,77,"apple,lemon,sugar,flour,cinnamon,nut,butter,sa..."
3,apple_pie/39_classic_apple_pie_hostedLargeUrl.jpg,88,"pie,apple,sugar,corn starch,cinnamon,lemon,but..."
4,apple_pie/13_mug_apple_pie_hostedLargeUrl.jpg,62,"apple,brown sugar,butter,cinnamon,allspice,nut..."
...,...,...,...
4821,chocolate_ice_cream/41_homemade_chocolate_ice_...,4817,"chocolate,egg,sugar,vanilla,chocolate,milk"
4822,chocolate_ice_cream/29_chocolate_ice_cream_hos...,4805,"whipping,condensed milk,chocolate"
4823,"chocolate_ice_cream/39_vanilla_bean,_white_cho...",4815,"milk,sugar,whipping,vanilla,beans,chocolate,or..."
4824,chocolate_ice_cream/13_homemade_chocolate_ice_...,4789,"condensed milk,milk,chocolate,cocoa,corn starc..."


In [44]:
# Split ingredients into individual items
data["ingredients"] = data["ingredients"].str.split(",")

# One-hot encode the 'ingredients' column
encoded_data = data["ingredients"].str.join("|").str.get_dummies()

# Concatenate the original DataFrame with the encoded DataFrame
encoded_data = pd.concat([data, encoded_data], axis=1)

In [45]:
encoded_data

,id,label,ingredients,half,abura age,aburage,achiote,adobo,agar,agave nectar,...,yellowfin tuna,yellowtail,yoghurt,yogurt,yolk,yuzu,yuzukosho,zaatar,zest,zucchini
0,apple_pie/20_homemade_apple_pie_hostedLargeUrl...,69,"[flour, salt, oil, cold water, apple, sugar, c...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,apple_pie/43_homestyle_apple_pie_hostedLargeUr...,91,"[shell, pie, sugar, flour, cinnamon, apple, le...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,apple_pie/28_apple_pie_hostedLargeUrl.jpg,77,"[apple, lemon, sugar, flour, cinnamon, nut, bu...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,apple_pie/39_classic_apple_pie_hostedLargeUrl.jpg,88,"[pie, apple, sugar, corn starch, cinnamon, lem...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple_pie/13_mug_apple_pie_hostedLargeUrl.jpg,62,"[apple, brown sugar, butter, cinnamon, allspic...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4821,chocolate_ice_cream/41_homemade_chocolate_ice_...,4817,"[chocolate, egg, sugar, vanilla, chocolate, milk]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4822,chocolate_ice_cream/29_chocolate_ice_cream_hos...,4805,"[whipping, condensed milk, chocolate]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4823,"chocolate_ice_cream/39_vanilla_bean,_white_cho...",4815,"[milk, sugar, whipping, vanilla, beans, chocol...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4824,chocolate_ice_cream/13_homemade_chocolate_ice_...,4789,"[condensed milk, milk, chocolate, cocoa, corn ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
encoded_data[["category", "image_name"]] = encoded_data["id"].str.split(
    "/", expand=True, n=1
)

In [47]:
encoded_data

,id,label,ingredients,half,abura age,aburage,achiote,adobo,agar,agave nectar,...,yoghurt,yogurt,yolk,yuzu,yuzukosho,zaatar,zest,zucchini,category,image_name
0,apple_pie/20_homemade_apple_pie_hostedLargeUrl...,69,"[flour, salt, oil, cold water, apple, sugar, c...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,apple_pie,20_homemade_apple_pie_hostedLargeUrl.jpg
1,apple_pie/43_homestyle_apple_pie_hostedLargeUr...,91,"[shell, pie, sugar, flour, cinnamon, apple, le...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,apple_pie,43_homestyle_apple_pie_hostedLargeUrl.jpg
2,apple_pie/28_apple_pie_hostedLargeUrl.jpg,77,"[apple, lemon, sugar, flour, cinnamon, nut, bu...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,apple_pie,28_apple_pie_hostedLargeUrl.jpg
3,apple_pie/39_classic_apple_pie_hostedLargeUrl.jpg,88,"[pie, apple, sugar, corn starch, cinnamon, lem...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,apple_pie,39_classic_apple_pie_hostedLargeUrl.jpg
4,apple_pie/13_mug_apple_pie_hostedLargeUrl.jpg,62,"[apple, brown sugar, butter, cinnamon, allspic...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,apple_pie,13_mug_apple_pie_hostedLargeUrl.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4821,chocolate_ice_cream/41_homemade_chocolate_ice_...,4817,"[chocolate, egg, sugar, vanilla, chocolate, milk]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,chocolate_ice_cream,41_homemade_chocolate_ice_cream_hostedLargeUrl...
4822,chocolate_ice_cream/29_chocolate_ice_cream_hos...,4805,"[whipping, condensed milk, chocolate]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,chocolate_ice_cream,29_chocolate_ice_cream_hostedLargeUrl.jpg
4823,"chocolate_ice_cream/39_vanilla_bean,_white_cho...",4815,"[milk, sugar, whipping, vanilla, beans, chocol...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,chocolate_ice_cream,"39_vanilla_bean,_white_chocolate_and_oreo_crum..."
4824,chocolate_ice_cream/13_homemade_chocolate_ice_...,4789,"[condensed milk, milk, chocolate, cocoa, corn ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,chocolate_ice_cream,13_homemade_chocolate_ice_cream_hostedLargeUrl...


## Train and test split

### Make sure for each category the train and test split is done equally

In [48]:
food_subset = encoded_data["category"].unique().tolist()

In [49]:
# Initialize lists to hold train and test data
train_data = []
test_data = []

# Iterate over each unique category
for category in food_subset:
    # Filter the DataFrame for the current category
    category_data = encoded_data[encoded_data["category"] == category]

    # Split the category data into train and test sets while ensuring equal representation
    train_cat, test_cat = train_test_split(
        category_data, test_size=0.1, random_state=42
    )

    # Add the train and test data to the respective lists
    train_data.append(train_cat)
    test_data.append(test_cat)

# Concatenate the train and test data for each category
train_df = pd.concat(train_data)
test_df = pd.concat(test_data)

In [50]:
# train_df["category"].unique()

In [51]:
# test_df["category"].unique()

In [52]:
"""
X = encoded_data["id"]  # Features
y = encoded_data.drop(["id", "label", "ingredients"], axis=1)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
"""

'\nX = encoded_data["id"]  # Features\ny = encoded_data.drop(["id", "label", "ingredients"], axis=1)\n\n# Split the data into training and testing sets (80% train, 20% test)\nX_train, X_test, y_train, y_test = train_test_split(\n    X, y, test_size=0.2, random_state=42\n)\n'

In [53]:
"""
train_df = pd.concat(
    [X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1
)
test_df = pd.concat(
    [X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1
)
"""

'\ntrain_df = pd.concat(\n    [X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1\n)\ntest_df = pd.concat(\n    [X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1\n)\n'

In [54]:
train_df = train_df.drop(["label", "ingredients", "category", "image_name"], axis=1)
train_df = train_df.reset_index(drop=True)
train_df

,id,half,abura age,aburage,achiote,adobo,agar,agave nectar,ahi,aioli,...,yellowfin tuna,yellowtail,yoghurt,yogurt,yolk,yuzu,yuzukosho,zaatar,zest,zucchini
0,apple_pie/35_apple_pie_hostedLargeUrl.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,apple_pie/12_magically_fast_apple_pie_hostedLa...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,apple_pie/21_szarlotka_(polish_apple_pie)_host...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,apple_pie/27_apple_pie_hostedLargeUrl.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple_pie/25_mom's_apple_pie_hostedLargeUrl.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4329,chocolate_ice_cream/42_magic_chocolate_ice_cre...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4330,chocolate_ice_cream/25_simply_perfect_chocolat...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4331,chocolate_ice_cream/45_chocolate_ice_cream_hos...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4332,chocolate_ice_cream/46_dark_chocolate_ice_crea...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
test_df = test_df.drop(["label", "ingredients", "category", "image_name"], axis=1)
test_df = test_df.reset_index(drop=True)
test_df

,id,half,abura age,aburage,achiote,adobo,agar,agave nectar,ahi,aioli,...,yellowfin tuna,yellowtail,yoghurt,yogurt,yolk,yuzu,yuzukosho,zaatar,zest,zucchini
0,apple_pie/26_apple_pie_hostedLargeUrl.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,apple_pie/29_traditional_apple_pie_hostedLarge...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,apple_pie/1_chai_apple_pie_hostedLargeUrl.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,apple_pie/2_perfect_apple_pie_hostedLargeUrl.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple_pie/9_brown_bag_apple_pie_hostedLargeUrl...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,chocolate_ice_cream/22_chocolate_ice_cream_hos...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
488,chocolate_ice_cream/43_chocolate_ice_cream_hos...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489,chocolate_ice_cream/12_caramelized_white_choco...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490,chocolate_ice_cream/41_homemade_chocolate_ice_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
target_val = test_df.columns.tolist()
target_val = target_val[1:]

## Creating model

In [57]:
bestmodel_path = "bestmodel_ingredient_efficientnetv2.keras"
trainedmodel_path = "trainedmodel_ingredient_efficientnetv2.keras"
history_path = "history_ingredient_efficientnetv2.log"

In [58]:
# Define data augmentation with more techniques
training_data = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.3,
    zoom_range=0.3,
    rotation_range=30,  # Add rotation augmentation
    width_shift_range=0.1,  # Add width shift augmentation
    height_shift_range=0.1,  # Add height shift augmentation
    horizontal_flip=True,
)

In [59]:
testing_data = ImageDataGenerator(preprocessing_function=preprocess_input)

In [60]:
training_data_generator = training_data.flow_from_dataframe(
    dataframe=train_df,
    directory="/Users/udyansachdev/Downloads/Recipes5k/images",
    x_col="id",
    y_col=target_val,
    target_size=(300, 300),
    batch_size=8,
    class_mode="raw",
)

Found 4334 validated image filenames.


In [61]:
validation_data_generator = testing_data.flow_from_dataframe(
    dataframe=test_df,
    directory="/Users/udyansachdev/Downloads/Recipes5k/images",
    x_col="id",
    y_col=target_val,
    target_size=(300, 300),
    batch_size=8,
    class_mode="raw",
)

Found 492 validated image filenames.


In [65]:
from tensorflow.keras.layers import (
    GlobalAveragePooling2D,
    Dense,
    Dropout,
    BatchNormalization,
)

# Define the model
inception = EfficientNetV2L(weights="imagenet", include_top=False)
fc = inception.output
fc = GlobalAveragePooling2D()(fc)
fc = Dense(128, activation="relu")(fc)
fc = Dropout(0.2)(fc)

predictions = Dense(
    1013, kernel_regularizer=regularizers.l2(0.005), activation="softmax"
)(fc)


model = Model(inputs=inception.input, outputs=predictions)
model.compile(
    optimizer=SGD(learning_rate=0.0001, momentum=0.9),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)

In [66]:
checkpoint = ModelCheckpoint(filepath=bestmodel_path, verbose=1, save_best_only=True)
csv_logger = CSVLogger(history_path)

In [ ]:
history = model.fit(
    training_data_generator,
    steps_per_epoch=training_data_generator.n // 8,
    validation_data=validation_data_generator,
    validation_steps=validation_data_generator.n // 8,
    epochs=20,
    verbose=1,
    callbacks=[csv_logger, checkpoint],
)

In [68]:
model.save(trainedmodel_path)

#### Loss Plotting

In [ ]:
def plot_training_history(history, title):
    plt.title(title)
    plt.plot(history.history["accuracy"], label="Training Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.legend(loc="best")
    plt.show()


def plot_loss_history(history, title):
    plt.title(title)
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(loc="best")
    plt.show()


plot_training_history(history, "Training and Validation Accuracy")
plot_loss_history(history, "Training and Validation Loss")

### Load Saved model

In [69]:
model_best = load_model("bestmodel_ingredient_efficientnetv2.keras", compile=False)

In [70]:
img = "/Users/udyansachdev/Downloads/ML_Final/test_images/3.jpeg"
img = image.load_img(img, target_size=(300, 300))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

In [71]:
pred = model_best.predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 71s 71s/step


In [73]:
pred

array([[0.00097396, 0.00102144, 0.0011225 , ..., 0.00095251, 0.0009871 ,
        0.00097644]], dtype=float32)

In [74]:
# Convert array to a PyTorch tensor
tensor = torch.tensor(pred)

# Get the indices of the top 10 maximum values
top10_indices = torch.topk(tensor, k=12).indices.tolist()

In [75]:
top10_indices[0]

[777, 617, 326, 997, 886, 669, 808, 724, 717, 658, 622, 238]

In [76]:
for i in top10_indices[0]:
    print(target_val[i])

salt
oil
egg
worcestershire
tahini
pig
shallot
radicchio
pumpkin
pepper
onion
condiments
